Based on *NEST Topology Module: A Case-Based Tutorial* (https://nest-simulator.readthedocs.io/en/nest-2.20.1/tutorials/topology/hill_tononi_Vp.html) and S. L. Hill and G. Tononi. Modeling Sleep and Wakefulness in the Thalamocortical System. _J Neurophysiology_ 93:1671-1698 (2005). Freely available via doi 10.1152/jn.00915.2004.

In [1]:
## Load useful packages
import numpy as np
import uuid
from scipy.spatial.distance import cdist

# ! Load pynest
import nest

# ! Load NEST Topoplogy module (NEST 2.2)
import nest.topology as topo

# ! Import math, we need Pi
import math

In [2]:
def run_spatial_ei(E_L_switch = -55.0):
    nest.ResetKernel()
    NUM_THREADS = 32 # set number of threads for parallelization
    nest.SetKernelStatus({"local_num_threads": NUM_THREADS})
    
    # ! Configurable Parameters
    # ! =======================
    # !
    # ! - Network size in neurons ``N``, each layer is ``N x N``.
    # ! - Network size in subtended visual angle ``visSize``, in degree.
    # ! - Background firing rate of retinal nodes ``retDC`` in Hz.
    # ! - Simulation duration ``simtime``; actual simulation is split into
    # !   intervals of ``sim_interval`` length, so that the network state
    # !   can be visualized in those intervals. Times are in ms.
    Params = {'N': 40,
              'visSize': 8.0,
              'retDC': 0.2, 
              'simtime': 60000.0,
              'sim_interval': 1.0,
              'init_interval' : 500.0
              }

    # ! Neuron Models
    # ! =============
    # !
    nest.CopyModel('iaf_psc_delta', 'NeuronModel',
                   params={'V_m': -70.0,
                           'E_L': -68.0, # Maldonado et al 2020
                           'C_m': 200.0, # Maldonado et al 2020
                           'tau_m': 84.0, # Maldonado et al 2020
                           't_ref': 2.0,
                           'V_th': -36.0, # Maldonado et al 2020
                           'V_reset': -60.0,
                           'I_e': 50.0
                           })

    # ! Adaptation of models for different populations
    # ! ----------------------------------------------

    # ! Cortical excitatory cells
    # ! .........................
    # ! Parameters are the same as above, so we need not adapt anything
    nest.CopyModel('NeuronModel', 'CtxExNeuron')

    # ! Cortical inhibitory cells
    # ! .........................
    nest.CopyModel('NeuronModel', 'CtxInNeuron',
                   params={'C_m': 200.0,
                           'tau_m': 106.6, # Maldonado et al 2020
                           'E_L' : -60.802, # Maldonado et al 2020
                           'V_th': -33.0, # Maldonado et al 2020
                           'I_e': 40.0
                          })


    # ! Input generating nodes
    # ! ----------------------

    nest.CopyModel('poisson_generator', 'RetinaNode',
                       params={'rate': Params['retDC']})
    
    nest.CopyModel('sinusoidal_poisson_generator', 'RetinaNodeSinus',
                   params={'rate': 0.05, 'amplitude': 0.075,
                           'frequency': 0.05, 'phase': 0.0})

    # ! Recording nodes
    # ! ---------------

    nest.CopyModel('multimeter', 'RecordingNode',
                   params={'interval': Params['sim_interval'],
                           'record_from': ['V_m'],
                           'record_to': ['memory'],
                           'withgid': True,
                           'withtime': False})

    # ! Populations
    # ! ===========
    
    # grid with jitter
    jit = 0.5
    xs = np.arange(-20,20,1)
    poss = [[x,y] for y in xs for x in xs]
    poss = [[p[0]+np.random.uniform(-jit,jit),p[1]+np.random.uniform(-jit,jit)] for p in poss]
    poss = (8*np.array(poss)/40).tolist()

    layerProps = {"positions": poss,
            "extent" : [Params['visSize']+1, Params['visSize']+1],
            'edge_wrap': True}
    # ! Retina
    # ! ------
    layerProps.update({'elements': 'RetinaNode'})
    retina = topo.CreateLayer(layerProps)
    
    layerProps.update({'elements': 'RetinaNodeSinus'})
    retinaSinus = topo.CreateLayer(layerProps)
    
    # ! We use list comprehesions to create all neuron types:
    nest.CopyModel('CtxExNeuron','L23pyr')
    nest.CopyModel('CtxInNeuron','L23in')

    layerProps.update({'elements': ['L23pyr', 2, 'L23in', 1]})
    Vp = topo.CreateLayer(layerProps)

    # ! Collect all populations
    # ! -----------------------

    # ! For reference purposes, e.g., printing, we collect all populations
    # ! in a tuple:
    populations = (retina , Vp) 

    # ! Synapse models
    # ! ==============

    nest.CopyModel('static_synapse', 'AMPA')
    nest.CopyModel('static_synapse', 'GABA_A')

    # ! Connections
    # ! ====================

    dpc = Params['visSize'] / (Params['N'] - 1)

    ccConnections = []

    horIntraBase = {"connection_type": "divergent",
                    "synapse_model": "AMPA",
                    "mask": {"circular": {"radius": 12.0 * dpc}},
                    "kernel": {"gaussian": {"p_center": 0.05, "sigma": 7.5 * dpc}},
                    "weights": 8.4, # 420 megaohm (palo) times 20 picoampere (Maldonado et al 2020)
                    "delays": {"uniform": {"min": 1.75, "max": 2.25}}}

    for conn in [{"sources": {"model": "L23pyr"}, "targets": {"model": "L23pyr"}},
                 {"sources": {"model": "L23pyr"}, "targets": {"model": "L23in"}}]:
        ndict = horIntraBase.copy()
        ndict.update(conn)
        ccConnections.append(ndict)

    intraInhBase = {"connection_type": "divergent",
                    "synapse_model": "GABA_A",
                    "mask": {"circular": {"radius": 7.0 * dpc}},
                    "kernel": {"gaussian": {"p_center": 0.25, "sigma": 7.5 * dpc}},
                    "weights": -21.3,# 533 megaohm (Palo) times 40 picoampere (Maldonado et al 2020)
                    "delays": {"uniform": {"min": 1.75, "max": 2.25}}}

    for conn in [{"sources": {"model": "L23in"}, "targets": {"model": "L23pyr"}},
                 {"sources": {"model": "L23in"}, "targets": {"model": "L23in"}}]:
        ndict = intraInhBase.copy()
        ndict.update(conn)
        ccConnections.append(ndict)

    # ! Cortico-cortical
    [topo.ConnectLayers(Vp, Vp, conn) for conn in ccConnections]

    retCort = {"connection_type": "divergent",
               "synapse_model": "AMPA",
               "mask": {"circular": {"radius": 1.0 * dpc}},
               "kernel": {"gaussian": {"p_center": 0.75, "sigma": 2.5 * dpc}},
               "weights": 8.4,
               "delays": 1.0}

    for conn in [{"targets": {"model": "L23pyr"}},
                 {"targets": {"model": "L23in"}}]:
        retCort.update(conn)
        topo.ConnectLayers(retina, Vp, retCort)

    for conn in [{"targets": {"model": "L23pyr"}}]:
        retCort.update(conn)
        topo.ConnectLayers(retinaSinus, Vp, retCort)
    
        
        
    nest.SetKernelStatus({'rng_seeds' : np.random.randint(1 , 1000000 , NUM_THREADS).tolist()})
    nest.SetStatus([0], {'print_time': True})

    nest.Simulate(Params['init_interval'])

    recorders = {}
    detectors = {}

    for name, loc, population, model in [('Vp L23pyr', 1, Vp, 'L23pyr'),
                                         ('Vp L23in', 2, Vp, 'L23in')]:

        population_leaves = nest.GetLeaves(population)[0]
        tgts = [nd for nd in population_leaves
                if nest.GetStatus([nd], 'model')[0] == model]
        recorders[name] = (nest.Create('RecordingNode' , n=len(tgts) ), loc)
        detectors[name] = (nest.Create('spike_detector'), loc)
        nest.Connect(recorders[name][0], tgts , {'rule' : 'one_to_one'})  # one recorders to one targets
        nest.Connect(tgts , detectors[name][0] , {'rule' : 'all_to_all'})  # one detector to all targets

    # ! show time during simulation
    nest.Simulate(Params['simtime'])
    ds = np.array([d['events']['V_m'] for d in nest.GetStatus(recorders['Vp L23pyr'][0])])
    ds_before_exc = ds[::2] # 0.5 * (ds[::2] + ds[1::2])
    ds_before_inh = np.array([d['events']['V_m'] for d in nest.GetStatus(recorders['Vp L23in'][0])])
    sps_before_exc = nest.GetStatus(detectors['Vp L23pyr'][0])[0]['events']
    sps_before_inh = nest.GetStatus(detectors['Vp L23in'][0])[0]['events']
    poss = (40*np.array(poss)/8)
    
    dMat_before = cdist(poss , poss,'euclidean')

    cMat_before = np.corrcoef(ds_before_exc)

    population_leaves = nest.GetLeaves(population , properties={'model' : 'L23in'})[0]
    nest.SetStatus(population_leaves[0::4] , {'E_L': E_L_switch}) # only change every 4th inhibitory unit, since SST is only ~23%
    nest.SetStatus(recorders['Vp L23pyr'][0], {'n_events': 0})
    nest.Simulate(Params['simtime'])
    ds = np.array([d['events']['V_m'] for d in nest.GetStatus(recorders['Vp L23pyr'][0])])
    ds_after_exc = ds[::2] # 0.5 * (ds[::2] + ds[1::2])
    ds_after_inh = np.array([d['events']['V_m'] for d in nest.GetStatus(recorders['Vp L23in'][0])])
    
    sps_after_exc = nest.GetStatus(detectors['Vp L23pyr'][0])[0]['events']
    sps_after_inh = nest.GetStatus(detectors['Vp L23in'][0])[0]['events']
    
    dMat_after = cdist(40*poss/8 , poss,'euclidean') 

    cMat_after = np.corrcoef(ds_after_exc)
        
    return (dMat_before ,ds_before_exc , ds_after_exc , sps_after_exc , sps_after_inh)

In [ ]:
MAXDIST = np.sqrt(20*20 + 20*20)
edges = np.linspace(0.1 , MAXDIST , 40)
u_tri_ind = np.triu_indices(40*40 , k=1)
accDMATs = []
accCMATs_before = []
accCMATs_after = []
REPS = 1
for xx in np.arange(REPS):
    if np.mod(xx , REPS/2) == 0:
        print(xx)
    (dMat_before ,ds_before_exc , ds_after_exc , sps_after_exc , sps_after_inh) = run_spatial_ei(E_L_switch = -56.3)
        
    unique_filename = str(uuid.uuid4())
    
    np.save('SIM_OUT/' + unique_filename + '_dMat_before.npy' , dMat_before)
    
    np.save('SIM_OUT/' + unique_filename + '_ds_before_exc.npy' , ds_before_exc)
    
    np.save('SIM_OUT/' + unique_filename + '_ds_after_exc.npy' , ds_after_exc)
            
    np.save('SIM_OUT/' + unique_filename + '_sps_after_times_exc.npy' , sps_after_exc['times'])
            
    np.save('SIM_OUT/' + unique_filename + '_sps_after_senders_exc.npy' , sps_after_exc['senders'])
            
    np.save('SIM_OUT/' + unique_filename + '_sps_after_times_inh.npy' , sps_after_inh['times'])
            
    np.save('SIM_OUT/' + unique_filename + '_sps_after_senders_inh.npy' , sps_after_inh['senders'])


0


/gpfs/gjor/data/Lab/kirchner/conda/envs/nest-env-openmpi/lib/python3.7/site-packages/nest/lib/hl_api_helper.py:127: UserWarning:
GetLeaves is deprecated and will be removed in NEST 3.0. Use GIDCollection instead.
